In [4]:
%load_ext yuuno
import havsfunc as haf
import os

The yuuno extension is already loaded. To reload it, use:
  %reload_ext yuuno


In [2]:
video_file= "test.avi"
src = core.lsmas.LWLibavSource(video_file)
src_10bit = core.fmtc.bitdepth(src, bits=10,fulls=False,fulld=False) # 提升到10bit
print(src)
print(src_10bit)

VideoNode
	Format: YUV420P8
	Width: 1920
	Height: 1080
	Num Frames: 3977
	FPS: 60
	Flags: NoCache IsCache

VideoNode
	Format: YUV420P10
	Width: 1920
	Height: 1080
	Num Frames: 3977
	FPS: 60
	Flags: NoCache IsCache



In [13]:
%%vspreview
src_10bit[:100].set_output() #预览

Preview(clip=<vapoursynth.VideoNode object at 0x00000198D968BF70>)

## %%vspipe 的基本用法
    %%vspipe [--outputindex <OUTPUT-ID>] [OPTIONS] | [<COMMAND>...]
    Options:
    --outputindex <OUTPUT-ID> - Which output-index should be encoded.
    --y4m - Add YUV4MPEG headers to output
    --start N - Set the output frame range (first frame)
    --end N - Set the output frame range (last frame)
    --requests N - Set the number of concurrent frame requests.
    --reset-core - Create a new vs.Core-object before executing the script.
    --isolate-core - Create a new vs.Core-object that is only running during the encode. Do not use with --reset-core
    --isolate-variables - Make sure that changes to variables do not affect the entire console.

In [14]:
%%vspipe --y4m | NVEncC64 -i - --vpy-mt --y4m -o defalut.mp4
src.set_output() #默认输出
#-i <input> 输入为pipe时 -
#--vpy-mt vapoursynth输入标记
#-o <output> 输出
"""
- <input> 输入为pipe时 -
--vpy-mt vapoursynth输入标记
-o <output> 输出
--y4m 将输入格式指定为 y4m (YUV4MPEG2)
"""

EncodeWidget(length=3977, start_time=1611578008)

例1: 3000-3999 帧使用vbrhq模式12000kbps编码、  
     5000-5999 帧使用固定质量29.0编码、  
     其他部分使用固定质量25.0编码。  
  --vbrhq 0 --vbr-quality=25.0 --dynamic-rc 3000:3999,vbrhq=12000 --dynamic-rc 5000:5999,vbrhq=0,vbr-quality=29.0  
例2: 3000 帧之前使用vbrhq模式6000kbps编码、  
     3000 帧之后使用vbrhq模式12000kbps编码。  
  --vbrhq 6000 --dynamic-rc start=3000,vbrhq=12000  

In [7]:
%%vspipe --y4m | NVEncC64 -i - --avhw --vpy-mt --y4m -c hevc --preset quality --vbrhq 0 --vbr-quality 29 --ref 16 --aq --aq-strength 7 --mv-precision Q-pel --lookahead 32 -o 8bit.mp4
src.set_output() #8bit
"""
--avhw 使用 avformat 和 cuvid 的硬件解码器。使用该模式可以提供最佳性能，因为该模式下整个编解码过程均在 GPU 运行
--preset quality  : performance, default, quality, P1, P2, P3, P4, P5, P6, P7
--vbrhq 0 --vbr-quality 29 2pass vbr 固定质量29
--aq 在帧内启用自适应量化（Adaptive Quantization）
--aq-temporal 在帧间启用自适应量化（Adaptive Quantization） H265不支持
--aq-strength 指定自适应量化强度（Adaptive Quantization Strength）。(1 (弱) - 15 (强), 0 = 自动)
--mv-precision 
auto ... 自动
Q-pel ... 1/4 像素精度 (高精确度)
half-pel ... 1/2 像素精度
full-pel ... 1 像素精度 (低精确度)

"""

EncodeWidget(length=3977, start_time=1611581508)

In [6]:
%%vspipe --y4m | NVEncC64 -i - --avhw --vpy-mt --y4m -c hevc --preset quality --vbrhq 0 --vbr-quality 40 --ref 16 --aq --aq-strength 7 --mv-precision Q-pel --lookahead 32 --output-depth 10 -o 10bit.mp4
src_10bit.set_output() # 10bit
"""
File "I:\conda\lib\site-packages\yuuno_ipython\ipy_vs\outputter.py", line 175, in encode
y4mformat = y4mformat + 'p' + str(self.format.bits_per_sample)
NameError: name 'self' is not defined
改成 y4mformat = y4mformat + 'p' + str(clip.format.bits_per_sample) 后
%reload_ext yuuno
"""

EncodeWidget(length=3977, start_time=1611580890)